In [104]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import tensorflow as tf

In [105]:
# Hyperparameters
split = 0.85
sequence_length = 10
epochs = 10
learning_rate = 0.02
selected_ticker = "META"

In [106]:
# Loading stock price data
stock_data = pd.read_csv("historical_data.csv")
columns = ['Close']
ticker_column = 'Ticker'

In [107]:
# Filter data for the selected ticker
stock_data = stock_data[stock_data[ticker_column] == selected_ticker]

In [108]:
# Encode the tickers
label_encoder = LabelEncoder()
stock_data['Ticker_Encoded'] = label_encoder.fit_transform(stock_data[ticker_column])

In [109]:
# Splitting data into train and test sets
stock_data['Split'] = stock_data.groupby('Ticker').cumcount() / stock_data.groupby('Ticker')['Date'].transform('count')
train_data = stock_data[stock_data['Split'] < split]
test_data = stock_data[stock_data['Split'] >= split]

In [110]:
# Normalize numerical data
scaler = MinMaxScaler()
train_data.loc[:, 'Close'] = scaler.fit_transform(train_data[['Close']])
test_data.loc[:, 'Close'] = scaler.transform(test_data[['Close']])

In [111]:
# Prepare sequences for training and testing
def prepare_sequences(data, sequence_length, scaler):
    X_seq, y = [], []
    group_values = data[columns].values
    for i in range(len(group_values) - sequence_length):
        X_seq.append(group_values[i:i + sequence_length])
        y.append(group_values[i + sequence_length])
    return np.array(X_seq), np.array(y)

In [112]:
X_train, y_train = prepare_sequences(train_data, sequence_length, scaler)
X_test, y_test = prepare_sequences(test_data, sequence_length, scaler)

In [113]:
# Model creation
def model_create():
    tf.random.set_seed(1234)
    
    # Sequential input for stock data
    seq_input = tf.keras.Input(shape=(X_train.shape[1], 1), name="Sequence_Input")
    
    # LSTM layers
    lstm_out = tf.keras.layers.LSTM(units=50, activation="tanh", return_sequences=True)(seq_input)
    lstm_out = tf.keras.layers.Dropout(0.15)(lstm_out)
    lstm_out = tf.keras.layers.LSTM(units=30, activation="tanh", return_sequences=True)(lstm_out)
    lstm_out = tf.keras.layers.Dropout(0.05)(lstm_out)
    lstm_out = tf.keras.layers.LSTM(units=20, activation="tanh", return_sequences=False)(lstm_out)
    lstm_out = tf.keras.layers.Dropout(0.01)(lstm_out)
    
    # Dense layers
    dense_out = tf.keras.layers.Dense(units=50, activation="relu")(lstm_out)
    dense_out = tf.keras.layers.Dropout(0.2)(dense_out)
    output = tf.keras.layers.Dense(units=1, activation="linear")(dense_out)
    
    # Compile the model
    model = tf.keras.models.Model(inputs=seq_input, outputs=output)
    model.compile(
        loss="mean_squared_error",
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    
    # Train the model
    model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=32,
        verbose=1
    )
    return model

In [114]:
# Invert normalization
y_test = scaler.inverse_transform(y_test)

In [115]:
# Prediction on test set
def predict(model):
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).reshape(-1, 1)
    return predictions

In [116]:
# Evaluation
def evaluate(predictions):
    mae = mean_absolute_error(predictions, y_test)
    mape = mean_absolute_percentage_error(predictions, y_test)
    return mae, mape, (1 - mape)

In [117]:
# Trial runs
def run_model(n):
    total_mae = total_mape = total_acc = 0
    for i in range(n):
        model = model_create()
        predictions = predict(model)
        mae, mape, acc = evaluate(predictions)
        total_mae += mae
        total_mape += mape 
        total_acc += acc 
    return (total_mae / n), (total_mape / n), (total_acc / n), predictions.tolist()

In [118]:
mae, mape, acc, preds = run_model(1)

print(f"Mean Absolute Error = {mae}")
print(f"Mean Absolute Percentage Error = {mape*100:.2f}%")
print(f"Accuracy = {acc}")

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.1339
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0196
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0119
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0077
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0083
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0071
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0061
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0055
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0057
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0056
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 368ms/step
Mean Absolute Error = 12.5035165640024
Mean Absolute Percentage Error = 3.96%
Accuracy = 0.9604161057375402
